In [5]:
import pandas as pd
import json
import numpy as np
import os

data_dir = "data/session-1/"
r_file="R2 2022-04-14 21_45_31.txt"
l_file="L2 2022-04-14 21_45_49.txt"
def processFile(filename):
    path = data_dir + filename
    log_data=open(path,'r')

    for line in log_data:
        print(line)

    # parse the log file into a pandas df
    headers = ["level", "timestamp", "message"]
    def log_to_json(_log_data, _headers):
        result={}
        i=0
        for line in _log_data:
            columns = line.split('\t') #or w/e you're delimiter/separator is
            data = {}
            for idx, c in enumerate(columns):
                # print(i, _headers[min(idx, len(_headers)- 1)], c)
                key = _headers[idx]
                value = c
                data[key] = value
            result[i] = data
            i += 1
        return json.dumps(result)

    df=pd.read_json(log_to_json(log_data, headers), orient='index')
    # select the application rows & parse the hex value representing the height
    df = df.loc[df['level'] == 'A']
    df['message'] = df.apply(lambda x: int(x['message'][9:11] + x['message'][6:8],16) if x['message'][1:5]=='(0x)' else np.NaN, axis = 1)   # magic one liner to parse the hexademical reperesnting the height (and time which I'm ignoring) sent by the sensor

    df.rename(columns = {'message':'dist'}, inplace = True)
    df = df.dropna()
    df = df.drop('level', axis=1)
    df = df.set_index('timestamp')
    return df

def processdf(session_id):
    # find files matching L + session_id & R + session_id
    l_df, r_df = None, None
    for file in os.listdir(data_dir):
        if file == 'L.txt':
            l_df = processFile(file)
        elif file == 'R.txt':
            r_df = processFile(file)
    assert(l_df is not None and r_df is not None)

    # find earliest common slow-time scan & lastest common scan & then drop rows not common on time
    earliest_common_scan = max(l_df.index[0], r_df.index[0])
    latest_common_scan = min(l_df.index[-1], r_df.index[-1])

    x, y = latest_common_scan > l_df.index, l_df.index > earliest_common_scan
    keeps = [a and b for a, b in zip(x, y)]
    index_to_drop = l_df.index[np.invert(keeps)]
    l_df.drop(index=index_to_drop, inplace=True)

    x, y = latest_common_scan > r_df.index, r_df.index > earliest_common_scan
    keeps = [a and b for a, b in zip(x, y)]
    index_to_drop = r_df.index[np.invert(keeps)]
    r_df.drop(index=index_to_drop, inplace=True)

    # average over 250ms windows & join df's
    l_df = l_df.reset_index()
    l_df['timestamp']= l_df.timestamp.dt.ceil(freq='50L')
    l_df = l_df.groupby('timestamp').mean()
    l_df = l_df.rename(columns={'dist':'dist_left'})

    r_df = r_df.reset_index()
    r_df['timestamp']= r_df.timestamp.dt.ceil(freq='50L')
    r_df = r_df.groupby('timestamp').mean()
    r_df = r_df.rename(columns={'dist':'dist_right'})

    df = pd.merge(l_df, r_df, how='left', left_index=True, right_index=True)
    df.ffill(inplace=True) # just incase a window has no samples
    return df
# %% pickle the data
df_1 = processdf(1)
df_1.to_pickle(data_dir + 'exercise_5')
# df_2 = processdf(2)
# df_2.to_pickle(data_dir + 'exercise_2')

nRF Connect, 2022-04-13

Calibrex v1.0.15 (E8:9F:6D:A4:6D:66)

I	13:57:12.245	[Server] Server started

V	13:57:12.249	Unknown Service (0000aaa0-0000-1000-8000-aabbccddeeff)

- Unknown Characteristic [N R] (0000aaa1-0000-1000-8000-aabbccddeeff)

   Client Characteristic Configuration (0x2902)

   Unknown Descriptor (0000aab0-0000-1000-8000-aabbccddeeff)

   Characteristic User Description (0x2901)

   Characteristic Presentation Format (0x2904)

- Unknown Characteristic [I W WNR] (0000aaa2-0000-1000-8000-aabbccddeeff)

   Client Characteristic Configuration (0x2902)

User Data (0x181C)

- First Name [R W] (0x2A8A)

- Last Name [R W] (0x2A90)

- Gender [R W] (0x2A8C)

V	13:57:12.249	Connecting to E8:9F:6D:A4:6D:66...

D	13:57:12.249	gatt = device.connectGatt(autoConnect = true, TRANSPORT_LE, preferred PHY = LE 1M)

D	16:54:01.618	[Server callback] Connection state changed with status: 0 and new state: DISCONNECTED (0)

I	16:54:01.618	[Server] Device disconnected

D	18:41:02.319	[Server c

KeyError: 'level'

In [ ]:
        # uploaded_files = list(map(lambda x: x.getvalue(), uploaded_files))
        # log_data = uploaded_files[0].read()
        # stringio = StringIO(uploaded_files[0].getvalue().decode('utf-8'))
        # uploaded_files[0]

        # log_data = log_data.split('\n')
        # for line in log_data:
        #     print(line)
        # for i, file in enumerate(uploaded_files):
        #     bytes_data = file.getvalue()
        #     st.write(bytes_data)
        #     dic[i] = 

        #     # bytes_data = [uploaded_files[0].read(), uploaded_files[1].read()]
        #     # stringio = StringIO(file.getvalue().decode('ISO-8859–1'))
        #     # string_data = stringio.read()
            
        #     # text_file = None
        #     # if i == 0:
        #     #     text_file = open("data/txt/L.txt", "w", encoding='ISO-8859–1')
        #     # else:
        #     #     text_file = open("data/txt/R.txt", "w", encoding='ISO-8859–1')

        #     # text_file.write(string_data)

        #     # text_file.close()
        
        # get_pickle("name", uploaded_files)